In [ ]:
using LaTeXStrings
using JLD
using Interact
using Plots; Plots.gr_font_family["bold"] = 11
using Plots.PlotMeasures
using StaticArrays
using TrafficWeavingPlanner
using DifferentialDynamicsModels
using LinearDynamicsModels

if isfile("config.json")
    config = JSON.parsefile("config.json")
else
    error("Please run setup.py in this directory before running any .ipynb's.")
end

hitl_save_dir = "HITL_samples/"

array_name = "trajectories_$(config["feature_extraction_dict"])"
include("traffic_weaving_control.jl")
xdd_choices = Float32[0, -3, 4, -6]
y_top, y_bot = y_targets = Float32[-1.83363, -6.09404]
tibvp = SteeringBVP(TripleIntegratorDynamics(1, Float32), TimePlusQuadraticControl(SMatrix{1,1}(Float32(1e-3))))
nothing

In [ ]:
function car_shapes(x = 0, y = 0, θ = 0; X = 4.22100019455, Y = 1.76199996471, dX = 1.3654999733, tD = 0.7, tW = 0.5)
    box(x, y) = Shape([(x/2,y/2), (-x/2,y/2), (-x/2,-y/2), (x/2,-y/2)])
    shapes = [
        translate!(box(X, Y), dX, 0.),
        translate!(box(tD, tW), 0., Y/2),
        translate!(box(tD, tW), 0., -Y/2),
        translate!(box(tD, tW), 2*dX, Y/2),
        translate!(box(tD, tW), 2*dX, -Y/2)
    ]
    translate!.(rotate!.(shapes, θ, ((0., 0.),)), x, y)
end

macro prediction_boilerplate(plot_code, name)
    quote
        Δt, PH, N, K = 0.1, 15, 2, 5
        z_unique = 1:K^N
        color_list = distinguishable_colors(3+length(z_unique), [RGB(.8,0.2,.8), RGB(1,0.5,0.25), RGB(0.3,.4,1)])[4:end]
        color_dict = Dict(z => c for (z,c) in zip(z_unique, color_list))
#         for fname in readdir("/home/schmrlng/ICRA18/")
#             println(fname)
        @manipulate for fname in readdir(hitl_save_dir)
            tmp_dict = load(hitl_save_dir * fname)
            car1 = tmp_dict["car1"]
            car2 = tmp_dict["car2"]
            extras = extra_features(car1, car2)
            T = size(car1, 2)
            anim = @animate for t in 1:size(car1, 2)
                if T - t < PH
                    lf = (car1[1,1,2] > -4 ? 2 : 1)
                    X, Y = all_future_xy(xdd_choices, y_targets, car1[1,end,:], tibvp, [(1,lf) for i in 1:PH-T+t], 0)
                    car1_future = zeros(1,PH,6)
                    car1_future[:,1:T-t,:] = car1[:,t+1:end,:]
                    car1_future[:,T-t+1:end,1:2:end] = X[:,2:end,:]
                    car1_future[:,T-t+1:end,2:2:end] = Y[:,2:end,:]
                else
                    car1_future = car1[:,t + (1:PH),:]
                end
                TrafficWeavingPlanner.predict!(car1, car2, extras, [t], car1_future, 50)

                pred_container = TrafficWeavingPlanner.pred_container
                x0, y0, xd0, yd0 = car2[1,t,1:4]
                xdd = hcat(fill(car2[1,t,5], size(pred_container.y,1)), pred_container.y[:,1,:,1])
                ydd = hcat(fill(car2[1,t,6], size(pred_container.y,1)), pred_container.y[:,1,:,2])
                xd = trapz(xdd, Δt, xd0)
                yd = trapz(ydd, Δt, yd0)
                x = trapz(xd, Δt, x0)
                y = trapz(yd, Δt, y0)
                z_list = Vector{Int}(pred_container.z[:,1,:]*[i*K^j for j in 0:N-1 for i in 0:K-1]) + 1
                colors = reshape([color_dict[z] for z in z_list], (1,length(z_list)))

                $plot_code
            end
#             mp4(anim, "/home/schmrlng/V_ICRA18/" * fname[1:end-3] * $name * ".mp4", fps = 10)
            gif(anim, $name * ".gif", fps = 10)
        end
#         "done!!"
    end
end

# FULL HIGHWAY XY

In [ ]:
@prediction_boilerplate begin
    plot([car1[1,1:t,1], car2[1,1:t,1]],
         [car1[1,1:t,2], car2[1,1:t,2]],
         xlims=(-150,15), ylims=(-8.3,0.3), aspectratio=:equal, size=(1600, 200), margin=1mm,
         legend=false,grid=true,yticks=[-8,-4,0],xticks=-150:50:50,linewidth=2,
         xaxis=font(20,"helvetica"),yaxis=font(20,"helvetica"))
    plot!([-170,80],[[0,0], [-4,-4], [-8,-8]], style=:dash, color=:grey, linewidth=2)
    plot!(x', y', linewidth=.5, alpha=0.2, color=colors, label="")
    for (i,s) in enumerate(car_shapes(car1[1,t,1], car1[1,t,2], atan2(car1[1,t,4], car1[1,t,3])))
        if i == 1
            plot!(s, color=:blue, label="")
#                 plot!(annotation=(mean(s.x),
#                                   mean(s.y),
#                                   text("R", font(20,:white))))
        else
            plot!(s, color=:black, label="")
        end
    end
    for (i,s) in enumerate(car_shapes(car2[1,t,1], car2[1,t,2], atan2(car2[1,t,4], car2[1,t,3])))
        if i == 1
            plot!(s, color=:red, label="")
#                 plot!(annotation=(mean(s.x),
#                                   mean(s.y),
#                                   text("H", font(20,:white))))
        else
            plot!(s, color=:black, label="")
        end
    end
end "full_xy"

# ZOOM XY

In [ ]:
@prediction_boilerplate begin
    plot([car1[1,1:t,1], car2[1,1:t,1]],
         [car1[1,1:t,2], car2[1,1:t,2]],
         xlims=(car1[1,t,1]-20,car1[1,t,1]+55),
         ylims=(-8.3,0.3), aspectratio=:equal, size=(1600, 300), margin=6mm, legend=false,
         yticks=[0,-4,-8], xticks=-170:10:80, xaxis=font(24), yaxis=font(24), linewidth=2)
    plot!([-170,80],[[0,0], [-4,-4], [-8,-8]], style=:dash, color=:grey, linewidth=2)
    plot!(x', y', linewidth=.5, alpha=0.2, color=colors, label="")
    for (i,s) in enumerate(car_shapes(car1[1,t,1], car1[1,t,2], atan2(car1[1,t,4], car1[1,t,3])))
        if i == 1
            plot!(s, color=:blue, label="")
            plot!(annotation=(mean(s.x),
                              mean(s.y),
                              text("R", font(20,:white,"bold"))))
        else
            plot!(s, color=:black, label="")
        end
    end
    for (i,s) in enumerate(car_shapes(car2[1,t,1], car2[1,t,2], atan2(car2[1,t,4], car2[1,t,3])))
        if i == 1
            plot!(s, color=:red, label="")
            plot!(annotation=(mean(s.x),
                              mean(s.y),
                              text("H", font(20,:white,"bold"))))
        else
            plot!(s, color=:black, label="")
        end
    end
end "zoom_xy"

# ZOOM XY (VERTICAL)

In [ ]:
@prediction_boilerplate begin
    y_tick_min = ceil(Int, (car1[1,t,1]-20)/10)*10
    y_tick_max = floor(Int, (car1[1,t,1]+55)/10)*10
    plot([car1[1,1:t,2], car2[1,1:t,2]],
         [car1[1,1:t,1], car2[1,1:t,1]],
         ylims=(car1[1,t,1]-20,car1[1,t,1]+55),
         xlims=(-8.3,0.3), aspectratio=:equal, size=(200, 800), legend=false,
         xticks=[0,-4,-8], yticks=(y_tick_min:10:y_tick_max, [rpad(string(yt), 5) for yt in y_tick_min:10:y_tick_max]),
         xaxis=font(16,"helvetica"), yaxis=font(16,"helvetica"), linewidth=2,
         ymirror=true, xflip=true, xlabel="t (m)", ylabel="s (m)", right_margin=10mm)
    plot!([[0,0], [-4,-4], [-8,-8]], [-170,80], style=:dash, color=:grey, linewidth=2)
    plot!(y', x', linewidth=.5, alpha=0.2, color=colors, label="")
    for (i,s) in enumerate(car_shapes(car1[1,t,2], car1[1,t,1], atan2(car1[1,t,3], car1[1,t,4])))
        if i == 1
            plot!(s, color=:blue, label="")
            plot!(annotation=(mean(s.x),
                              mean(s.y),
                              text("R", font(12,:white,"bold"))))
        else
            plot!(s, color=:black, label="")
        end
    end
    for (i,s) in enumerate(car_shapes(car2[1,t,2], car2[1,t,1], atan2(car2[1,t,3], car2[1,t,4])))
        if i == 1
            plot!(s, color=:red, label="")
            plot!(annotation=(mean(s.x),
                              mean(s.y),
                              text("H", font(12,:white,"bold"))))
        else
            plot!(s, color=:black, label="")
        end
    end
end "zoom_xy_vert"

# ZOOM XDD

In [ ]:
@prediction_boilerplate begin
    T_history = max(1,t-10):t
    T_future = t:min(t+PH, T)
    car1_history = car1[1,T_history,:]
    car2_history = car2[1,T_history,:]
    car1_future = car1[1,T_future,:]
    car2_future = car2[1,T_future,:]

    val(cA) = acc(cA[:,3], cA[:,5], cA[:,4], cA[:,6])
    val_f = acc(xd, xdd, yd, ydd)
    c1h = val(car1_history)
    c2h = val(car2_history)
    c1f = val(car1_future)
    c2f = val(car2_future)

    plot([Δt*(T_history - 1), Δt*(T_future - 1), Δt*(T_history - 1), Δt*(T_future - 1)],
         [c1h, c1f, c2h, c2f],
         linewidth=2,
         color=[:blue :blue :red :red],
         linestyles=[:solid :dash :solid :dash],
         xlims=(Δt*(t-10-1), Δt*(t+PH-1)),
         xticks=0:10,
         ylims=(-8,8),
         yticks=[-5,0,5],
         margin=3mm,
         ylabel="Longitudinal Acceleration -\nBody Frame (m/s^2)",
         left_margin=10mm,
#              titlefont=font(14,"Courier"),
         legend=false, xlabel="Time (s)",
         xaxis=font(16,"helvetica"), yaxis=font(16,"helvetica"))
    plot!(Δt*((t:t+PH) - 1), val_f', linewidth=.5, alpha=0.5, color=colors, label="")
end "zoom_xdd"

# ZOOM XD

In [ ]:
@prediction_boilerplate begin
    T_history = max(1,t-10):t
    T_future = t:min(t+PH, T)
    car1_history = car1[1,T_history,:]
    car2_history = car2[1,T_history,:]
    car1_future = car1[1,T_future,:]
    car2_future = car2[1,T_future,:]

    val(cA) = vel(cA[:,3], cA[:,4])
    val_f = vel(xd, yd)
    c1h = val(car1_history)
    c2h = val(car2_history)
    c1f = val(car1_future)
    c2f = val(car2_future)

    plot([Δt*(T_history - 1), Δt*(T_future - 1), Δt*(T_history - 1), Δt*(T_future - 1)],
         [c1h, c1f, c2h, c2f],
         linewidth=2,
         color=[:blue :blue :red :red],
         linestyles=[:solid :dash :solid :dash],
         xlims=(Δt*(t-10-1), Δt*(t+PH-1)),
         ylims=(20,36),
         xticks=0:10,
         yticks=16:4:40,
         margin=3mm,
         ylabel="Longitudinal Velocity -\nBody Frame (m/s)",
         left_margin=10mm,
#              titlefont=font(14,"Courier"),
         legend=false, xlabel="Time (s)",
         xaxis=font(16,"helvetica"), yaxis=font(16,"helvetica"))
    plot!(Δt*((t:t+PH) - 1), val_f', linewidth=.5, alpha=0.5, color=colors, label="")
end "zoom_xd"

# 4096

In [ ]:
anim = @animate for y in y_top+.5:-0.2:y_bot-.5
    println(y)
    pfX, pfY = all_future_xy(xdd_choices, y_targets, Float32[-150, y, 30, 0, 0, 0], tibvp, [(1,1), (1,1), (1,1)], 4)
    plot(repeat(pfX[:,:,1], inner=[size(pfY,1),1])', repeat(pfY[:,:,1], outer=[size(pfX,1),1])',
         color=:blue, legend=false, alpha=0.05)

    pfX, pfY = all_future_xy(xdd_choices, y_targets, Float32[-100, y, 30, 1, 0, -1], tibvp, [(1,2), (1,2), (1,2)], 4)
    plot!(repeat(pfX[:,:,1], inner=[size(pfY,1),1])', repeat(pfY[:,:,1], outer=[size(pfX,1),1])',
         color=:blue, legend=false, alpha=0.05)

    pfX, pfY = all_future_xy(xdd_choices, y_targets, Float32[-50, y, 30, -2, 0, 0], tibvp, [(1,2), (1,2), (1,2)], 4)
    plot!(repeat(pfX[:,:,1], inner=[size(pfY,1),1])', repeat(pfY[:,:,1], outer=[size(pfX,1),1])',
         color=:blue, legend=false, alpha=0.05)

    plot!([-150, 0], [[0,0], [-4,-4], [-8,-8]],
        color=:grey, linewidth=2, style=:dash, ylim=(-8.3, 0.3),
        xticks=-150:25:0,
        yticks = [0,-4,-8],
        xaxis=font(16,"helvetica"), yaxis=font(16,"helvetica"),
        xlabel="Longitudinal coordinate s (m)",
        ylabel="Lateral coordinate t (m)",
        grid=false, size=(800,400))
end

gif(anim, "4096.gif", fps = 20)
# mp4(anim, "/home/schmrlng/4096.mp4", fps = 20)